This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules

import json
import requests



In [2]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-11-20&end_date=2017-11-20&api_key= '
r = requests.get(url)
json_d = r.json()

print(json_d)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2018-11-21T22:32:56.759Z', 'newest_available_date': '2018-11-21', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-11-20', 'end_date': '2017-11-20', 'data': [['2017-11-20', 46.57, 47.38, 46.54, 47.04, None, 97252.0, 4563515.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


In [3]:
print(json_d['dataset']['column_names'])
print(json_d['dataset']['data'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2017-11-20', 46.57, 47.38, 46.54, 47.04, None, 97252.0, 4563515.0, None, None, None]]


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 

In [4]:
# Assign URL to variable

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key= '
r = requests.get(url)


### 2. Convert the returned JSON object into a Python dictionary.

In [13]:
# Decoding the JSON data into a dictionary json_data

json_data = r.json()

json_cols = json_data['dataset']['column_names']
json_dt = json_data['dataset']['data']


** The relevant data for the analysis is found under the json_data dictionary key 'data' which is a list of lists
where each list corresponds to each day**


In [92]:
print(json_cols)
print(json_dt[0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:

def hilo_opening(stklist):
    '''This function takes in a list of lists as argument and compares the second element of each list to
        to the previous max and min values which updated as you move from one list to next'''
    max_price = stklist[0][1]
    min_price = stklist[0][1]
    for x in stklist:
        if x[1] is not None:
            if x[1] > max_price:
                max_price=x[1]
            if x[1] < min_price:
                min_price = x[1]
    return max_price, min_price
  
x, y = hilo_opening(json_dt)

print('The highest opening price is $',x, 'and the lowest opening price is $',y)


The highest opening price is $ 53.11 and the lowest opening price is $ 34.0


### 4. What was the largest change in any one day (based on High and Low price)?

In [9]:
def hilomax_daily(stklist):
    max_change = 0
    change_date = ''
    for x in stklist:
        if (x[2] is not None) and (x[3] is not None):
            
            if x[2]-x[3] > max_change:
                max_change = x[2]-x[3]
                change_date= x[0]
    return change_date, max_change 

u, v = hilomax_daily(json_dt)

print('The largest change of any one day was $', v,'that occured on',u)

The largest change of any one day was $ 2.8100000000000023 that occured on 2017-05-11


### 5. What was the largest change between any two days (based on Closing Price)?

In [10]:
def closemax_btwndays(stklist):
    max_change = 0
    mxchange_date1 =''
    mxchange_date2 =''
    for i in range(len(stklist)-1):
        if stklist[i][4] - stklist[i+1][4] > max_change:
            max_change = stklist[i][4] - stklist[i+1][4]
            mxchange_date1 = stklist[i][0]
            mxchange_date2 = stklist[i+1][0]
            
            
    return mxchange_date1,mxchange_date2,max_change

date1 , date2, change = closemax_btwndays(json_dt)

print('The largest change between two consecutive days was $',change,' between the ',date1,' and ',date2)



The largest change between two consecutive days was $ 1.7199999999999989  between the  2017-05-11  and  2017-05-10


### 6. What was the average daily trading volume during this year?

In [11]:
def tradevol_avg(stklist):
    sum_vol = 0
    for x in stklist:
        sum_vol = sum_vol + x[6]
    vol_avg = sum_vol/len(stklist)
            
            
    return vol_avg 

print( 'The average daily trading volume during the year 2017 was',tradevol_avg(json_dt),'shares')

The average daily trading volume during the year 2017 was 89124.33725490196 shares


### 7. What was the median trading volume during this year

In [12]:
def tradevol_med(stklist):
    vol_list = []
    
    for x in stklist:
        vol_list.append(x[6])
        vol_list.sort()
    if len(vol_list)%2 == 0:
        midval = len(vol_list)/2
        vol_med = (vol_list[midval-1]+vol_list[midval])/2
    else:
        vol_med = vol_list[(len(vol_list)-1)//2]
           
    return vol_med 

print( 'The Median trading volume during the year 2017 was',tradevol_med(json_dt),'shares')

The Median trading volume during the year 2017 was 76286.0 shares
